In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy
from collections import defaultdict
import importlib
import pprint

from interval import interval

import screed

import alignplot
importlib.reload(alignplot)

from alignplot import StackedDotPlot
from alignplot import AlignmentDiagram

import numpy as np

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

SyntaxError: invalid syntax (alignplot.py, line 106)

In [ ]:
if 1:
    dotplot = StackedDotPlot('GCA_003222275.1',
                             ['GCA_003220225.1'],
                             'list.csv', './genomes', use_mashmap=True)

In [ ]:
_ = dotplot()

In [ ]:
dotplot = StackedDotPlot('GCF_001683825.1', ('GCF_000181835.1',), 'list.csv', './genomes',)

In [ ]:
import alignplot
importlib.reload(alignplot)

alignme = AlignmentDiagram(dotplot)
alignme.calculate()
fig = alignme.plot(use_labels=True)
fig.set_size_inches(10,8)
plt.savefig('/tmp/foo.png')

In [ ]:
alignme.calculate(plot_all_contigs=True)
fig = alignme.plot(use_labels=False)
fig.set_size_inches(10,8)
plt.savefig('/tmp/foo2.png')